In [1]:
#importing libraries
import pandas as pd
import sqlite3
import matplotlib
import seaborn

In [2]:
#configuring mysql into jupyter
%load_ext sql
%config SqlMagic.autopandas=True

In [3]:
#converting the statement PDF file to a data frame
import tabula

#path to the PDF file
pdf_path = "/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf"

# password to unlock the PDF file
password = ""

# options for the PDF extraction
options = {
    "pages": "all",        # Extracting data from all the pages
    "password": password   # password to unlock the PDF
}

# read tables from a PDF file and save them as CSV files
#tabula.convert_into('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.pdf', 'mpesa_statement.csv', output_format='csv', **options)

#had initially converted to a csv file
#changed it to an excel file to load the data better into a database table
#.ods is the same as .xlsx

df = pd.read_excel('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement.ods')

#dropping the unnamed column
#df = df.drop('Unnamed', axis=1)

# Print the extracted data
print(df)


                      Receipt No.   
0                      REF8MUC88I  \
1                      REF8MUC88I   
2                      REF9MU2IWF   
3                      REF9MU2IWF   
4                      REF7MT5G7N   
...                           ...   
3985                   QEG03RDEQM   
3986                   QEG23RDHRU   
3987  Statement Verification Code   
3988                          NaN   
3989                     KZ3FXNRZ   

                                        Completion Time   
0                                   2023-05-15 20:19:22  \
1                                   2023-05-15 20:19:22   
2                                   2023-05-15 20:17:21   
3                                   2023-05-15 20:17:21   
4                                   2023-05-15 20:10:35   
...                                                 ...   
3985                                2022-05-16 12:52:30   
3986                                2022-05-16 12:52:29   
3987  To verify the vali

In [4]:
#creating a local database to store the data
conn = sqlite3.connect('mpesa_statement_database')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS mpesa (
    `Receipt No.` VARCHAR,
    `Completion Time` DATETIME,
    Details VARCHAR,
    `Transaction Status` VARCHAR,
    `Paid In` INT,
    Withdrawn INT,
    Balance INT
)
''')
conn.commit()

In [5]:
df.to_sql('mpesa', conn, if_exists='replace', index = False)

3990

In [6]:
import os
from sqlalchemy import create_engine
# Construct the connection URL
db_url = f"sqlite:///{os.path.abspath('/home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database')}"

# Set the DATABASE_URL environment variable
os.environ['DATABASE_URL'] = db_url

# Create a database engine
engine = create_engine(os.environ['DATABASE_URL'])

In [7]:
%%sql
SELECT * FROM mpesa;

Done.


,Receipt No.,Completion Time,Details,Transaction Status,Paid In,Withdrawn,Balance
0,REF8MUC88I,2023-05-15 20:19:22,Customer Transfer Fuliza MPesato - 07******212...,Completed,None,-50,0
1,REF8MUC88I,2023-05-15 20:19:22,OverDraft of Credit Party,Completed,50,None,50
2,REF9MU2IWF,2023-05-15 20:17:21,Merchant Payment Fuliza M-Pesato 7384394 - DAM...,Completed,None,-30,0
3,REF9MU2IWF,2023-05-15 20:17:21,OverDraft of Credit Party,Completed,30,None,30
4,REF7MT5G7N,2023-05-15 20:10:35,Customer Transfer Fuliza MPesato - 2547******5...,Completed,None,-10,0
...,...,...,...,...,...,...,...
3985,QEG03RDEQM,2022-05-16 12:52:30,OD Loan Repayment to 232323 -M-PESA Overdraw,Completed,None,-480.37,569.63
3986,QEG23RDHRU,2022-05-16 12:52:29,Funds received from -07******245 daniel njeri,Completed,1050,None,1050
3987,Statement Verification Code,To verify the validity of this M-PESA statemen...,None,None,None,None,None
3988,None,None,None,None,None,None,None


**Showing the number of times i used fuliza in the past year**

In [8]:
%%sql
SELECT COUNT(Details) AS `Number of Times used Fuliza`, SUM(ABS(`Paid In`)) AS `Total Amount Used`
FROM mpesa WHERE Details
LIKE '%Overdraft%';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Number of Times used Fuliza,Total Amount Used
0,924,68083.82


**showing the number of times fuliza was used to pay for alcohol**

In [17]:
%%sql
SELECT COUNT(Details) AS 'Number of times',
SUM(ABS(Withdrawn)) AS `Total amount used`
FROM mpesa WHERE Details
LIKE '%Fuliza%' AND Details LIKE '%WINES%';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Number of times,Total amount used
0,53,22315.0


**Showing total amount spent on alcohol**

In [16]:
%%sql
SELECT COUNT(Details), SUM(ABS(Withdrawn)) AS 'Total amount used to pay for alcohol'
FROM mpesa
WHERE Details LIKE '%Wines%' OR Details LIKE '%SIMBA' OR Details LIKE '%PUB';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,COUNT(Details),Total amount used to pay for alcohol
0,90,37389.0


**Showing the total amount used to repay Fuliza**

In [11]:
%%sql
SELECT SUM(ABS(Withdrawn)) AS `Amount used to repay fuliza loan`
FROM mpesa WHERE Details
LIKE '%Fuliza%';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Amount used to repay fuliza loan
0,104549.0


**Showing amount used to buy airtime**

In [19]:
%%sql
SELECT SUM(ABS(Withdrawn)) AS `Total amount used`
FROM mpesa WHERE Details
LIKE '%Airtime%';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Total amount used
0,3318.0


**Amount used to buy recreational drugs**

In [21]:
%%sqlE '%%' OR Det
SELECT SUM(ABS(Withdrawn)) AS `Total Amount Used`
FROM mpesa
WHERE Details LIKails LIKE '%%' OR Details LIKE '%%'; -- Hiding the names you know why

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Total Amount Used
0,7325.0


**Showing total amount received**

In [25]:
%%sql
SELECT SUM(`Paid In`) AS `Total Amount Received`
FROM mpesa
WHERE Details LIKE '%Funds received%' OR Details LIKE '%International Transfer%';

 * sqlite:////home/muregz/data_analysis/mpesa_statement_analysis/mpesa_statement_database
Done.


,Total Amount Received
0,223348
